In [1]:
import ogr, gdal
import os
import pandas as pd
path = os.getcwd()

In [2]:
driver = ogr.GetDriverByName("ESRI Shapefile")

In [3]:
xzqhshp = path +"/china/"+ "cn.shp"
outfile = path + "/china.shp"

### gdal.SetConfigOption("SHAPE_ENCODING", "CP936")
### 这句话需要放在copy之前，或者是创建shapefile之前，否则会出现乱码

In [4]:
xzqh = driver.Open(xzqhshp)
gdal.SetConfigOption("SHAPE_ENCODING", "CP936")
cp = driver.CopyDataSource(xzqh, outfile)
xzqh.Destroy()
cp.Release()

In [5]:
outputfile = path + "/china_10000.shp"

In [6]:
ds = driver.Open(xzqhshp)
if os.access( outputfile, os.F_OK ):
    driver.DeleteDataSource( outputfile )
layer = ds.GetLayer()
layer.SetAttributeFilter("GDP_2005 > 10000")
gdal.SetConfigOption("SHAPE_ENCODING", "CP936")
newds = driver.CreateDataSource(outputfile)
cpl  = newds.CopyLayer(layer, 'abcd')
newds.Destroy()

In [27]:
print(outputfile)
dataSource = driver.Open(outputfile, 0)
layer = dataSource.GetLayer()
lyd = layer.GetLayerDefn()
fields = [lyd.GetFieldDefn(i).GetName() for i in range(lyd.GetFieldCount())]
layer.ResetReading()
feDict = []
for feature in layer:
    f = {}
    for field in fields:
        val = feature.GetField(field)
        if isinstance(val,str):
            val = val.encode('gbk').decode('utf-8')
        f[field] = val
    geo = feature.GetGeometryRef()
    f["Shape"] = geo.ExportToWkt()
    feDict.append(f)
    feature.Destroy()
layer.ResetReading()
dataSource.Destroy()
pd.DataFrame(feDict)

D:\workspace\DevWork\PyExample\exam4/china_10000.shp


,AREA,CATEGORIES,CPI_2009,Code,FIRST_NAME,GDP_2005,GDP_2006,GDP_2007,GDP_2008,GDP_2009,Pop_1990,Pop_1995,Pop_1999,Pop_2000,Pop_2005,Pop_2009,Pop_Birth_,Pop_Death_,Shape
0,1.003209e+11,2.0,11993.0,320000,江苏,18598.69,21742.05,26018.48,30981.98,34457.30,6705.6519,7066.0,7213.0,7438.0,7475.0,7725.0,9.55,6.56,"POLYGON ((119.78779842 34.4813239650001,120.29..."
1,1.009434e+11,2.0,15790.0,330000,浙江,13417.68,15718.47,18753.73,21462.69,22990.35,4144.5930,4319.0,4475.0,4677.0,4898.0,5180.0,10.22,6.75,"POLYGON ((120.12826222 30.947050775,120.516654..."
2,1.542384e+11,2.0,10494.0,370000,山东,18366.87,21900.19,25776.91,30933.28,33896.65,8439.2827,8705.0,8883.0,9079.0,9248.0,9470.3,11.70,6.47,"POLYGON ((122.214118076 36.907911525,122.24096..."
3,1.650694e+11,2.0,6607.0,410000,河南,10587.42,12362.79,15012.46,18018.53,19480.46,8550.9535,9100.0,9387.0,9256.0,9380.0,9487.0,11.45,6.28,"POLYGON ((114.523150095 36.174611795,114.90826..."
4,1.760488e+11,3.0,15291.0,440000,广东,22557.37,26587.76,31777.01,36796.71,39482.56,6282.9236,6868.0,7270.0,8642.0,9194.0,9638.0,11.78,5.70,"POLYGON ((114.25986752 25.2930331750001,114.39..."
5,1.855880e+11,1.0,7193.0,130000,河北,10012.11,11467.60,13607.32,16011.97,17235.48,6108.2439,6437.0,6614.0,6744.0,6851.0,7034.4,12.93,6.32,MULTIPOLYGON (((118.91233351064 39.12308411298...
